### Importing the libraries

In [1]:
import keras
import cv2
import mediapipe as mp
import numpy as np
from keras.preprocessing.image import img_to_array
import imutils
import os
import pickle

### Loading the model

In [2]:
model = keras.models.load_model("ArSLModelv2.h5")
file = open("ArSLLabels.pkl", "rb")
labels = pickle.load(file)

In [8]:
labels

{0: 'ain',
 1: 'al',
 2: 'aleff',
 3: 'bb',
 4: 'dal',
 5: 'dha',
 6: 'dhad',
 7: 'fa',
 8: 'gaaf',
 9: 'ghain',
 10: 'ha',
 11: 'haa',
 12: 'jeem',
 13: 'kaaf',
 14: 'khaa',
 15: 'la',
 16: 'laam',
 17: 'meem',
 18: 'nun',
 19: 'ra',
 20: 'saad',
 21: 'seen',
 22: 'sheen',
 23: 'ta',
 24: 'taa',
 25: 'thaa',
 26: 'thal',
 27: 'toot',
 28: 'waw',
 29: 'ya',
 30: 'yaa',
 31: 'zay'}

In [9]:
labels = {
    0: 'ع',
    1: 'ال',
    2: 'أ',
    3: 'ب',
    4: 'د',
    5: 'ذ',
    6: 'ض',
    7: 'ف',
    8: 'ق',
    9: 'غ',
    10: 'ه',
    11: 'ح',
    12: 'ج',
    13: 'ك',
    14: 'خ',
    15: 'لا',
    16: 'ل',
    17: 'م',
    18: 'ن',
    19: 'ر',
    20: 'ص',
    21: 'س',
    22: 'ش',
    23: 'ط',
    24: 'ت',
    25: 'ث',
    26: 'ذ',
    27: 'ة',
    28: 'و',
    29: 'ئ',
    30: 'ي',
    31: 'ز'
}

In [3]:
def classify(image):
    # Image Preprocessing
    image = cv2.resize(image, (64, 64))
    image = image.astype("float") / 255.0
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    
    # Predicting the class
    proba=model.predict(image)
    idx = np.argmax(proba)
    
    return labels[idx]

### Live Camera

In [4]:
cap = cv2.VideoCapture(0)

while True:
    ret, img = cap.read()
    img = cv2.flip(img, 1)
    
    top, right, bottom, left = 75, 350, 300, 590
    
    roi = img[top:bottom, right:left]
    roi=cv2.flip(roi,1)
    
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (7, 7), 0)
    
    kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
    gray = cv2.filter2D(gray, -1, kernel)
    
    cv2.imshow('roi', gray)
    
    pred = classify(gray)
    cv2.rectangle(img, (left, top), (right, bottom), (0,255,0), 2)
    font=cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(img, pred, (0,130), font, 5, (0,0,255), 2)
    
    cv2.imshow('img',img)
    
    key = cv2.waitKey(1) & 0xFF
    if key==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

### Live V3

In [5]:
import arabic_reshaper
from bidi.algorithm import get_display
from PIL import ImageFont, ImageDraw, Image


def putArabicText(frame, text, p):
    reshaped_text = arabic_reshaper.reshape(text)
    bidi_text = get_display(reshaped_text) 
    fontpath = "arial.ttf" 
    font = ImageFont.truetype(fontpath, 60)
    img_pil = Image.fromarray(frame)
    draw = ImageDraw.Draw(img_pil)
    draw.text(p, bidi_text, font = font, fill=(0, 0, 0))
    img = np.array(img_pil)
    
    return img

In [ ]:
import mediapipe as mp

mpHands = mp.solutions.hands
hands = mpHands.Hands(max_num_hands=1)
mpDraw = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

while True:
    ret, img = cap.read()
    img = cv2.flip(img, 1)
    h, w, c = img.shape
    
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = hands.process(imgRGB)
    hand_landmarks = result.multi_hand_landmarks
    
    frame = img.copy()
    if hand_landmarks:
        for handLMs in hand_landmarks:
            x_max = 0
            y_max = 0
            x_min = w
            y_min = h
            
            for lm in handLMs.landmark:
                x, y = int(lm.x * w), int(lm.y * h)
                if x > x_max:
                    x_max = x
                if x < x_min:
                    x_min = x
                if y > y_max:
                    y_max = y
                if y < y_min:
                    y_min = y
            
            x_min = int(x_min*0.9)
            y_min = int(y_min*0.6)
            x_max = int(x_max*1.1)
            y_max = int(y_max*1.1)
            
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 255), 2)
            mpDraw.draw_landmarks(frame, handLMs, mpHands.HAND_CONNECTIONS)
            top, right, bottom, left = y_min, x_min, y_max, x_max
    
    
    roi = img[top:bottom, right:left]
    roi=cv2.flip(roi,1)
    
    if not hand_landmarks:
        roi = np.array([])
        cv2.imshow('img', frame)
    
    if roi.size > 0:
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (7, 7), 0)

        kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
        gray = cv2.filter2D(gray, -1, kernel)

        cv2.imshow('roi', gray)

        pred = classify(gray)
        #cv2.rectangle(frame, (left, top), (right, bottom), (0,255,0), 2)
        font=cv2.FONT_HERSHEY_SIMPLEX
        cv2.rectangle(frame,(x_min-1, y_min+10), (x_max+1, y_min-50), (0, 255, 255), -2)
        #cv2.putText(frame, pred, (x_min, y_min), font, 2, (0 ,0 ,0), 2)
        frame = putArabicText(frame, pred, (x_min, y_min-50))
        cv2.imshow('img', frame)
    
    key = cv2.waitKey(1) & 0xFF
    if key==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

In [28]:
#pip install arabic_reshaper
#pip install python-bidi